# DNN 설정시 주의사항

- Sequantial()내부 첫번째 Dense()는 input_shape=.. 필수  
- Dense()의 unit수는 임의로 정함.  
  
1. 이진분류(0,1)  
- Sequantial()내부 히든 Dense()의 activation="relu", unit수는 임의로  
- Sequantial()내부 마지막 Dense()의 activation="sigmoid", unit수는 1  
- compile()내 loss="binary_crossentropy"  
- predict() 는 1이 될 확률 반환  
  
2. 다항분류  
- Sequantial()내부 히든 Dense()의 activation="relu", unit수는 임의로  
- Sequantial()내부 마지막 Dense()의 activation="softmax", unit수는 종류갯수  
- compile()내 loss="sparse_categorical_crossentropy"  
- predict() 는 1이 될 확률이 가장 큰 순번 반환(0 시작)  
  
3. 회귀  
- Sequantial()내부 마지막 Dense()의 activation 생략(linear기본), unit수는 1  
- compile()내 loss="mse" mean_squared_error  
- predict() 는 1이 될 확률 반환  

In [1]:
# 타이타닉 생존율 예측
# 데이터로딩
from tensorflow.keras import models, layers
import pandas as pd
import seaborn as sns

ti = sns.load_dataset("titanic")
print(ti.columns)

# X,y 정의
y = ti["survived"]
X = ti[["pclass","age","sibsp","parch"]]

# X의 결측치 처리
X["age"] = X["age"].fillna(X["age"].mean())

# X의 표준화(평균을 0)/정규화(0~1)
# 표준화 : 각 칼럼이 독립적일 경우
# 정규화 : 모든 칼럼이 동일한 의미의 정보를 가지는 경우(ex 픽셀정보)
from  sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_ss = ss.fit_transform(X)

# 훈련, 테스트데이터 분리
from sklearn.model_selection import train_test_split
x_tr, x_te, y_tr, y_te = train_test_split(
    X_ss, y, random_state=100, stratify=y
)
# 모델 생성
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu', input_shape=(x_tr.shape[1],)),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# model.summary()
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# 모델 훈련
model.fit(x_tr, y_tr, epochs=5000, verbose=0)
model.evaluate(x_te, y_te)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


C:\Users\admin\AppData\Local\Temp\ipykernel_14628\4209251764.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["age"] = X["age"].fillna(X["age"].mean())


7/7 [==============================] - 0s 1ms/step - loss: 0.5956 - accuracy: 0.7040


[0.5955715179443359, 0.7040358781814575]

In [ ]:
# 타이타닉 생존률 예측
from tensorflow.keras import models, layers, Sequential
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd

# 데이터 로딩
ti = sns.load_dataset('titanic')

# X,y 정의
X = ti.drop('survived',axis=1)
y = ti['survived']

# X의 결측치 처리
X['age'] = X['age'].fillna(X['age'].mean())
X['embark_town'] = X['embark_town'].fillna(X['embark_town'].mode())
X['embarked'] = X['embarked'].fillna(X['embarked'].mode())
X.isna().sum()

# 훈련, 테스트데이터 분리
x_tr, x_te, y_tr, y_te = train_test_split(
    X, y, random_state=100, stratify=y
)

# 모델 생성
model = Sequential([
    layers.Dense(4, activation='relu', input_shape=(x_tr.shape[1],)),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# model.summary()
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# 모델 훈련
model.fit(x_tr, y_tr, epochs=5000, verbose=0)
model.evaluate(x_te, y_te)